In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Import data
feature_path = "../data/train_features.csv"
test_path = "../data/test_features.csv"
label_path = "../data/train_labels.csv"

X = pd.read_csv(feature_path)
X_test = pd.read_csv(test_path)
y = pd.read_csv(label_path)

# Covert to time series df
def to_timeseries(df: pd.DataFrame, colnames: List[str], time_steps: int, unique_id: str):
    """Coverts dataframe into time series version with unique identifier"""
    X_new = pd.DataFrame()
    
    unique_ids = df.loc[:, unique_id].unique()
    print("unique_ids:", len(unique_ids))

    #for id_ in unique_ids:
    patient_rows = df[df[unique_id] == unique_ids[0]]
    
    #for index, row in patient_rows.iterrows():
    #    X_new[index, ""]
     
to_timeseries(X, X.columns, 12, "pid")
#### After ts

ts_path = "../data/data_p.csv"
X_ts = pd.read_csv(ts_path)

In [ ]:
data = pd.read_csv("../data/train_features.csv")
d = data.shape[1]

data_1hours = []
data_p = pd.DataFrame()
for i in range(12):
    id_ith = range(i,len(data),12)
    data_1hour = data.iloc[id_ith].iloc[:, [0] + [_ for _ in range(3, d)]]
    
    columns = data_1hour.columns
    columns_new = ["pid"]
    
    # Construct new columns
    for j in range(1,len(columns)):
        col_new = columns[j] + "-" + str(i)
        columns_new.append(col_new)
    data_1hour.columns = columns_new
    
    data_1hours.append(data_1hour)

    if data_p.empty:
        data_p = data_1hour
    else:
        data_p = pd.merge(data_p,data_1hour,how="inner",on="pid") 
data_p
data_p.to_csv("../data/train_features_transformed.csv")

In [ ]:
def new_data(row):
    return np.concatenate((
        np.sum(np.isnan(row) + 0, axis=0), 
        12 - np.sum(np.isnan(row) + 0, axis=0),
        np.nanmean(row, axis=0),
        np.nanmax(row, axis=0),
        np.nanmin(row, axis=0),
        np.nanmedian(row, axis=0),
        np.nanstd(row, axis=0),
        np.nanquantile(row, 0.9, axis=0)
    ))

In [ ]:
d = pd.read_csv("/Users/simondanielsson/Documents/ETH/intro-to-ml/data_joschi/full_feature_set.csv", header=None)
path_features = "../data/test_features.csv"
path_labels = "../data/train_labels.csv"

raw_data = pd.read_csv(path_features)
training_labels = pd.read_csv(path_labels)

# Initialize New DataFrame By Columns

raw_columns = list(raw_data.columns)
used_columns = raw_columns
not_used_columns = ['pid', 'Age', 'Time']

for name in not_used_columns:
    used_columns.remove(name)

columns_max = used_columns.copy()
for i, name in enumerate(columns_max):
    columns_max[i] = columns_max[i] + '_max'

columns_min = used_columns.copy()
for i, name in enumerate(columns_min):
    columns_min[i] = name + '_min'

columns_med = used_columns.copy()
for i, name in enumerate(columns_med):
    columns_med[i] = name + '_med'

features = ['pid', 'age'] + columns_min + columns_max + columns_med

feature_set = pd.DataFrame(columns=features)

ini_df = np.zeros((raw_data['pid'].nunique(), feature_set.shape[1]))
feature_set = feature_set.append(pd.DataFrame(ini_df, columns=feature_set.columns), ignore_index=True)

# Fill DataFrame by Grouped Patient Data

grouped_features = raw_data.groupby("pid")

index_var = 0
for name, data in grouped_features:

    feature_set.iloc[index_var,0] = name
    feature_set.iloc[index_var, 1] = data.iloc[0, 2]
    feature_set.iloc[index_var, 2:36] = grouped_features.get_group(name).min().iloc[3:]
    feature_set.iloc[index_var, 36:70] = grouped_features.get_group(name).max().iloc[3:]
    feature_set.iloc[index_var, 70:104] = grouped_features.get_group(name).median().iloc[3:]
    
    index_var = index_var + 1

In [ ]:
def get_feature(df):
    data = df[:,3]
    new_data_ = data.reshape((12, np.around(data.shape[1] * data.shape[0] / 12)))    

    training_data = np.array(new_data)
    for index, _ in enumerate(np.around(df.shape[0]/12)):
        training_data[index,:] = new_data(data[12*i: 12+12*i, ::])

    return training_data

X_train = get_feature(X)
X_test = get_feature(X_test)
pd.DataFrame(X_train).to_csv('X_train_stat.csv', index=False)
pd.DataFrame(X_test).to_csv('X_test_stat.csv', index=False) 

In [ ]:
# Compute proportion of missing values
tot_rows = feature_set.shape[0]
prop_missing_vals = {}
missing_values = 0


for col_name in feature_set.columns:
    missing_vals = np.sum(feature_set.loc[:, col_name].isna())
    missing_values = missing_values + missing_vals
    prop_missing_vals[col_name] = missing_vals / tot_rows

    #if prop_missing_vals[col_name] > 0.8:
    #    X.drop(col_name, inplace=True, axis=1)
#print(missing_values)
def sort_dict(d: dict) -> dict:
    return {k: v for k, v in sorted(d.items(), key=lambda item: item[1])}
    
prop_missing_vals = sort_dict(prop_missing_vals)
#print(prop_missing_vals)

props = list(prop_missing_vals.values())
plt.plot(props)
plt.suptitle("Proportion of missing values per feature", size=16)
plt.xlabel("Feature index (sorted order)")
plt.ylabel("Proportion of values missing")